In [ ]:
import geopandas as gpd

dem_file = config['dem']
shp = config['peilgebieden']
data = gpd.read_file(shp)

for index, row in data.iterrows():
    streefpeil = row['streefpeil']
    print(streefpeil)

#peilgebieden shapefile to raster
meta=hrt.create_meta_from_gdf(gdf=pgb_shp, res=dem.metadata['pixel_width'])
hrt.gdf_to_raster(gdf=pgb_shp, value_field='peil_id', raster_out=str(output_folder + '\combined1.tiff'),
    nodata=0, metadata=meta, epsg=28992, driver='GTiff')


In [ ]:
def define_lowest_point(rasterfile):
    rasterfile = rasterfile 
    percentiles = [5] 
    rasterArray = gdal_array.LoadFile(rasterfile) #Read raster as numpy array
    rasterArray = np.where(rasterArray==-9999, np.nan, rasterArray)

    for p in percentiles:
        lowest_1 = np.nanpercentile(rasterArray,p)
        print(lowest_1)
    return lowest_1


In [ ]:
#make raster of shapefile
shapefile = config['peilgebieden'] #CHANGE
output_folder = config['output_folder']

def peilgebieden_raster(shapefile, output_folder):
   
    #making the shapefile as an object.
    input_shp = ogr.Open(shapefile)

    #getting layer information of shapefile.
    shp_layer = input_shp.GetLayer()

    #pixel_size determines the size of the new raster.
    #pixel_size is proportional to size of shapefile.
    pixel_size = 0.5

    #get extent values to set size of output raster.
    x_min, x_max, y_min, y_max = shp_layer.GetExtent()

    #calculate size/resolution of the raster.
    x_res = int((x_max - x_min) / pixel_size)
    y_res = int((y_max - y_min) / pixel_size)

    #get GeoTiff driver by 
    image_type = 'GTiff'
    driver = gdal.GetDriverByName(image_type)

    #set name
    output_raster = str(output_folder + '\combined.tiff')
   
    
    #passing the filename, x and y direction resolution, no. of bands, new raster.
    new_raster = driver.Create(output_raster, x_res, y_res, 1, gdal.GDT_Float64, options=['COMPRESS=LZW'])

    #transforms between pixel raster space to projection coordinate space.
    new_raster.SetGeoTransform((x_min, pixel_size, 0, y_min, 0, pixel_size))

    #get required raster band.
    band = new_raster.GetRasterBand(1)

    #assign no data value to empty cells.
    no_data_value = -9999
    band.SetNoDataValue(no_data_value)
    band.FlushCache()

    #main conversion method
    gdal.RasterizeLayer(new_raster, [1], shp_layer, burn_values=[255], options=["ATTRIBUTE=id"])

    #adding a spatial reference
    new_rasterSRS = osr.SpatialReference()
    new_rasterSRS.ImportFromEPSG(28992)
    new_raster.SetProjection(new_rasterSRS.ExportToWkt())
    return gdal.Open(output_raster)

peilgebieden_raster(shapefile, output_folder)